This Notebook evaluates a convergence study for the Bow Shock Problem as presented presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [1]:
#r "BoSSSpad.dll"
#r "XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [2]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_ConvStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESF_BowShock_ConvStudy'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESF_BowShock_ConvStudy'.


{ Session Count = 12; Grid Count = 12; Path = \\dc3\userspace\sebastian\cluster\XESF_BowShock_ConvStudy }

In [3]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-04-03 13:44:05Z

In [4]:
var sessions = database.Sessions;
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0*	5/27/2024 1:21:20 PM	8e8c5db2...
#1: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0*	5/27/2024 1:20:57 PM	c6cfb32b...
#2: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0*	5/27/2024 1:20:36 PM	9e3cc0b3...
#3: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0*	5/27/2024 1:20:11 PM	21cec51c...
#4: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 3:12:58 PM	13d0abde...
#5: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:20:52 PM	705aac94...
#6: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:43:57 PM	c3bc10a4...
#7: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:16:31 PM	fe883d23...
#8: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:14:05 PM	cab2c050...
#9: XDGBowShock_TwoLs	XDGBS-p3-10x32

In [5]:
var aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

8e8c5db2-d164-4e09-a11e-b64ead5a58ae

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(\gamma -1)(\rho E - \rho e_{kin}),~\rho e_{kin}=\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock ($p_\text{in}=1,\rho_\text{in}=1$) presented, the v-velocity is zero and we have
$$ h_{in} = \rho E_{in}+1$$
We prescribe the inflow conditions by the Mach number $M_{in}=4$. From those the Energy is computed as 
$$ \rho E_{in} = (\rho e_{inner})_\text{in}+(\rho e_{kin})_\text{in}=\frac{p_\text{in}}{\gamma-1}+\frac{\rho_\text{in}}{2}u_\text{in}^2= \frac{1}{\gamma-1}+\frac{1}{2}u_{in}^2$$ 
where $u_{in} = M_{in}\sqrt{\gamma}$. In total we obtain:
$$ h_\text{in}=\rho E_{in}+1=\frac{\gamma}{\gamma-1}+\frac{1}{2}u_{in}^2$$


In [6]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

14.7

In [7]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [28]:
var sessions=database.Sessions.Where(s => (s.Name.Contains("iFlx0") &&s.Name.Contains("iPrb0")&&!s.Name.Contains("40x128") && s.SuccessfulTermination)); // the one plotted in the publication
sessions.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
sessions


#0: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:43:57 PM	c3bc10a4...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:06 PM	990fe52f...
#2: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:05 PM	6ec2412f...


# A Helper Function to obtain different plot formats

In [29]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Enthalpy Error Plot


In [30]:
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 160 
 
 
 
 
 180 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M511.2,57.1 L564.6,57.1 M45.6,132.3 L51.1,134.3 L56.6,135.7 L62.2,138.0 L67.7,146.6 L73.2,158.3
 L78.7,162.2 L84.3,163.8 L89.8,166.9 L95.3,173.1 L100.8,182.3 L106.3,182.1 L111.9,182.4 L117.4,182.4
 L122.9,182.3 L128.4,182.3 L134.0,182.3 L139.5,182.3 L145.0,182.2 L150.5,182.2 L156.1,182.2 L161.6,182.2
 L167.1,182.2 L172.6,182.2 L178.1,182.3 L183.7,182.3 L189.2,182.3 L194.7,182.3 L200.2,182.3 L205.8,182.2
 L211.3,182.2 L216.8,182.2 L222.3,182.3 L227.8,182.3 L233.4,182.3 L238.9,182.3 L244.4,182.3 L249.9,182.3
 L255.5,182.3 L261.0,182.3 L266.5,182.3 L272.0,182.3 L277.5,182.3 L283.1,182.3 L288.6,182.3 L294.1,182.3
 L299.6,182.3 L305.2,182.3 L310.7,182.3 L316.2,182.3 L321.7,182.3 L327.2,182.3 L332.8,182.3 L338.3,182.3
 L343.8,182.3 L349.3,182.3 L354.9,182.3 L360.4,182.3 L365.9,199.2 L371.4,213.0 L377.0,229.5 L382.5,229.7
 L388.0,229.8 L393.5,245.4 L399.0,265.2 L404.6,290.0 L410.1,347.9 L415.6,371.7 L421.1,375.4 L426.7,375.6
 L432.2,375.6 L437.7,375.7 L443.2,375.6 L448.7,375.6 L454.3,375.7 L459.8,375.7 L465.3,375.7 L470.8,375.6
 L476.4,375.7 L481.9,375.7 L487.4,375.7 L492.9,375.7 L498.4,375.7 L504.0,375.7 L509.5,375.7 L515.0,375.7
 L520.5,375.7 L526.1,375.7 L531.6,375.7 L537.1,375.7 L542.6,375.7 L548.1,375.7 L553.7,375.7 L559.2,375.7
 L564.7,375.7 L570.2,375.7 L575.8,375.7 L581.3,375.7 L586.8,375.7 L592.3,375.7 L597.9,375.7 L603.4,375.7
 L608.9,375.7 L614.4,375.7 L619.9,375.7 L625.5,375.7 L631.0,375.7 L636.5,375.7 L642.0,375.7 L647.6,375.7
 L653.1,375.7 L658.6,375.7 L664.1,375.7 L669.6,375.7 L675.2,375.7 L680.7,375.7 L686.2,375.7 L691.7,375.7
 L697.3,375.7 L702.8,375.7 L708.3,375.7 L713.8,375.7 L719.3,375.7 L724.9,375.7 L730.4,375.7 L735.9,375.7
 L741.4,375.7 L747.0,463.6 L752.5,464.2 L758.0,472.1 L763.5,472.1 L769.0,472.3 L774.6,472.2 L780.1,472.4
 L785.6,472.4 L791.1,472.4 L796.7,472.4 L802.2,472.4 L807.7,472.4 L813.2,472.4 L818.8,472.4 L824.3,472.4
 L829.8,472.4 L835.3,472.4 L840.8,472.4 L846.4,472.4 L851.9,472.4 L857.4,472.4 L862.9,472.4 L868.5,472.4
 L874.0,472.4 L879.5,472.4 L885.0,472.4 L890.5,472.4 L896.1,472.4 L901.6,472.4 L907.1,472.4 L912.6,472.4
 L918.2,472.4 L923.7,472.4 L929.2,472.4 L934.7,472.4 L940.2,472.4 L945.8,472.4 L951.3,472.4 L956.8,472.4
 L962.3,472.4 L967.9,472.4 L973.4,472.4 L978.9,472.4 L984.4,472.4 L989.9,472.4 L995.5,472.4 L1001.0,472.4
 L1006.5,472.4 L1012.0,472.4 L1017.6,472.4 L1023.1,472.4 L1028.6,472.4 L1034.1,472.4 L1039.7,541.2 L1045.2,540.6
 L1050.7,540.6 L1056.2,540.7 L1061.7,540.7 L1067.3,540.7 L1072.8,540.8 L1078.3,540.7 L1083.8,540.7 L1089.4,540.7
 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M511.2,81.1 L564.6,81.1 M45.6,146.1 L51.1,130.9 L56.6,134.9 L6

## Residual History

In [31]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1600,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 160 
 
 
 
 
 180 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M822.4,57.1 L875.8,57.1 M45.6,92.6 L53.1,95.7 L60.6,97.0 L68.2,98.8 L75.7,108.8 L83.2,121.7
 L90.7,133.0 L98.3,134.8 L105.8,123.7 L113.3,133.0 L120.8,147.5 L128.3,200.8 L135.9,226.8 L143.4,215.0
 L150.9,248.8 L158.4,257.4 L166.0,252.4 L173.5,260.2 L181.0,257.4 L188.5,263.5 L196.0,262.5 L203.6,267.3
 L211.1,256.7 L218.6,265.3 L226.1,271.8 L233.7,273.0 L241.2,276.0 L248.7,279.9 L256.2,281.2 L263.8,287.3
 L271.3,289.7 L278.8,289.6 L286.3,296.1 L293.8,298.0 L301.4,298.8 L308.9,301.0 L316.4,301.4 L323.9,302.3
 L331.5,302.9 L339.0,303.5 L346.5,304.8 L354.0,304.9 L361.5,305.3 L369.1,305.3 L376.6,308.7 L384.1,308.7
 L391.6,308.7 L399.2,309.1 L406.7,311.2 L414.2,311.6 L421.7,311.7 L429.2,311.8 L436.8,311.9 L444.3,311.9
 L451.8,311.9 L459.3,311.9 L466.9,311.9 L474.4,311.9 L481.9,102.3 L489.4,114.4 L496.9,122.3 L504.5,124.1
 L512.0,124.2 L519.5,125.6 L527.0,131.5 L534.6,157.8 L542.1,197.1 L549.6,230.0 L557.1,271.3 L564.7,285.9
 L572.2,299.4 L579.7,307.4 L587.2,322.0 L594.7,325.6 L602.3,330.8 L609.8,330.7 L617.3,334.1 L624.8,335.5
 L632.4,339.0 L639.9,342.6 L647.4,344.1 L654.9,345.7 L662.4,346.6 L670.0,348.5 L677.5,349.4 L685.0,349.7
 L692.5,349.8 L700.1,350.0 L707.6,351.4 L715.1,352.7 L722.6,353.3 L730.1,353.2 L737.7,353.7 L745.2,353.8
 L752.7,355.0 L760.2,355.0 L767.8,355.7 L775.3,356.1 L782.8,356.6 L790.3,356.7 L797.8,356.6 L805.4,356.8
 L812.9,356.9 L820.4,357.2 L827.9,357.5 L835.5,357.7 L843.0,357.9 L850.5,358.3 L858.0,358.0 L865.6,358.3
 L873.1,358.6 L880.6,359.1 L888.1,359.1 L895.6,359.2 L903.2,359.2 L910.7,359.4 L918.2,359.4 L925.7,359.4
 L933.3,359.4 L940.8,359.5 L948.3,359.5 L955.8,359.6 L963.3,359.7 L970.9,359.7 L978.4,359.7 L985.9,359.7
 L993.4,359.7 L1001.0,186.8 L1008.5,270.8 L1016.0,309.7 L1023.5,334.9 L1031.0,343.7 L1038.6,353.3 L1046.1,372.9
 L1053.6,376.1 L1061.1,380.4 L1068.7,390.0 L1076.2,394.3 L1083.7,392.6 L1091.2,399.1 L1098.7,399.8 L1106.3,400.2
 L1113.8,404.8 L1121.3,404.7 L1128.8,406.7 L1136.4,411.0 L1143.9,411.3 L1151.4,411.5 L1158.9,412.5 L1166.5,413.2
 L1174.0,414.4 L1181.5,414.6 L1189.0,414.7 L1196.5,415.2 L1204.1,415.2 L1211.6,415.4 L1219.1,415.7 L1226.6,416.6
 L1234.2,416.8 L1241.7,416.8 L1249.2,416.9 L1256.7,417.1 L1264.2,417.1 L1271.8,417.0 L1279.3,417.4 L1286.8,417.6
 L1294.3,417.7 L1301.9,418.0 L1309.4,417.9 L1316.9,417.5 L1324.4,417.6 L1331.9,418.0 L1339.5,418.2 L1347.0,418.4
 L1354.5,418.3 L1362.0,418.4 L1369.6,418.4 L1377.1,418.4 L1384.6,418.5 L1392.1,418.4 L1399.6,332.8 L1407.2,358.0
 L1414.7,376.1 L1422.2,377.1 L1429.7,390.8 L1437.3,396.0 L1444.8,393.7 L1452.3,400.3 L1459.8,397.2 L1467.4,400.6
 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-

save results to text file

In [32]:
public void SavePlotTable(List<double> Iterations, List<double> y1, string l1, List<double> y2 = null, string l2 = null, List<double> y3 = null, string l3 = null) {
            var table1 = MultidimensionalArray.Create(y1.Count,2);
            table1.SetColumn(0, Iterations);
            table1.SetColumn(1, y1);
            table1.SaveToTextFile(l1 + ".txt");
            if(y2 != null && l2 != null) {
                var table2 = MultidimensionalArray.Create(y2.Count, 2);
                table2.SetColumn(0, Iterations);
                table2.SetColumn(1, y2);
                table2.SaveToTextFile(l2 + ".txt");
            }
            if(y3 != null && l3 != null) {
                var table3 = MultidimensionalArray.Create(y3.Count, 2);
                table3.SetColumn(0, Iterations);
                table3.SetColumn(1, y3);
                table3.SaveToTextFile(l3 + ".txt");
            }
}
var si= sessions.Pick(0);
IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
var enthalpyerrors = new List<double>();
foreach(var timestep in si.Timesteps){
    var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
    enthalpyerrors.Add(error);
}
SavePlotTable(lts.TimeStepNumbers,lts.EnResHistory, "EnResNorms", lts.ResHistory, "ResNorms", enthalpyerrors, "EnthalpyErrors")

## Convergence Plot Enthalpy

choose the sessions

they must be reordered for the plot to work...

In [33]:
var ses= sessions;
ses = ses.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
ses

#0: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:05 PM	6ec2412f...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:06 PM	990fe52f...
#2: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:43:57 PM	c3bc10a4...


Number of cells

In [34]:
int[] Cells = ses.Select(si => si.GetGrids().Pick(0).NumberOfCells).ToArray();
Cells

[ 80, 320, 1280 ]

helper arrays

In [47]:
int[] degs = new int[] {0,1,2,3};
var errors_array= MultidimensionalArray.Create(4,ses.Count()+1);


Function used to calculate the EOCs

In [48]:
double ApproxSlope(List<double> errors, List<double> cellSize) {
    var slopes = new List<double>();
    double EOC;
    for (int i = 0; i < errors.Count - 1; i++) {
        double ratio = errors[i] / errors[i + 1];
        double slope = Math.Log(ratio) / Math.Log(cellSize[i] / cellSize[i + 1]);
        slopes.Add(slope);
    }
    if (slopes.Count > 0) {
        EOC = slopes.Average();
    } else {
        EOC = 0;
    }
    return EOC;
}

get ideal plot

In [49]:
List<double> GetIdealConvPlot(List<double> errors,List<double> cellSize, int p){
        var idealErrors = new List<double>();

    // Calculate ideal errors for each cell size
    for (int i = 0; i < cellSize.Count; i++)
    {
        double idealError = errors[0] * Math.Pow(cellSize[i] / cellSize[0], p + 1);
        idealErrors.Add(idealError);
    }

    return idealErrors;
} 

function for different plot formats

In [50]:
public PlotFormat GetFormat(int count){
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

do the convergence plot

In [62]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();
int count=0;
MultidimensionalArray errorsAndCells=MultidimensionalArray.Create(ses.Count()+1,5);
MultidimensionalArray errorsAndCellsIdeal=MultidimensionalArray.Create(ses.Count()+1,5);
foreach(int p in degs){
    var cells= new List<double>();
    var errors= new List<double>();
    for(int iSess =0;iSess < ses.Count();iSess++){
        var si = ses.Pick(iSess);
        //extract the relevant field 
        var timesteps = si.Timesteps;
        var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == p);
        if(rTSIndex !=-1){
            IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)timesteps.Pick(rTSIndex)).GetInternal();
            var fields = lts.Fields.ToList();
            var enthalpyError = (XDGField) lts.GetField("h_err").Clone();
            errors_array[p,iSess]=(enthalpyError.L2NormAllSpecies()/EEN);
            cells.Add(Math.Sqrt(enthalpyError.Basis.GridDat.iGeomCells.GetCellVolume(0)));
            errors.Add(enthalpyError.L2NormAllSpecies()/EEN);
            Console.WriteLine($"P={p}, h={Math.Sqrt(enthalpyError.Basis.GridDat.iGeomCells.GetCellVolume(0))} ---> error={enthalpyError.L2NormAllSpecies()/EEN}");
        }
    }
    errorsAndCells.SetColumn(0,cells);
    errorsAndCellsIdeal.SetColumn(0,cells);
    errorsAndCells.SetColumn(p+1,errors);
    errorsAndCellsIdeal.SetColumn(p+1,GetIdealConvPlot(errors,cells,p));
    EOCs.Add( ApproxSlope(errors,cells));
    plot.AddDataGroup($"P={p}",cells,errors, GetFormat(count));
    var fmt = GetFormat(count);
    fmt.PointSize=0.01;
    fmt.DashType=DashTypes.Dashed;

    plot.AddDataGroup($"ideal {p+1}",cells,GetIdealConvPlot(errors,cells,p), fmt);
    count++;
}
errorsAndCells.SaveToTextFile("ConvergenceStudyErrors.txt");
errorsAndCellsIdeal.SaveToTextFile("ConvergenceStudyErrorsIdeal.txt");
plot.Xlabel="Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:500)

P=0, h=0.44721359549995787 ---> error=0.0239408559855892
P=0, h=0.22360679774997894 ---> error=0.013459432541759641
P=0, h=0.11180339887498954 ---> error=0.007003204505232315
P=1, h=0.44721359549995787 ---> error=0.0032144940105414323
P=1, h=0.22360679774997894 ---> error=0.0006497499506191621
P=1, h=0.11180339887498954 ---> error=0.0002075701090410999
P=2, h=0.44721359549995787 ---> error=0.0008750016444413794
P=2, h=0.22360679774997894 ---> error=0.00012598447060909864
P=2, h=0.11180339887498954 ---> error=3.5726850023537516E-05
P=3, h=0.44721359549995787 ---> error=0.0003778250186566546
P=3, h=0.22360679774997894 ---> error=3.481094613539085E-05
P=3, h=0.11180339887498954 ---> error=1.0309590300542094E-05
Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Cells 
 
 
 
 
 P=0 
 
 
 P=0 
 
 
 
 
 
 
 
 
 
 ideal 1 
 
 
 ideal 1 
 
 
 
 
 
 
 
 
 
 P=1 
 
 
 P=1 
 
 
 
 
 
 
 
 
 
 ideal 2 
 
 
 ideal 2 
 
 
 
 
 
 
 
 
 
 P=2 
 
 
 P=2 
 
 
 
 
 
 
 
 
 
 ideal 3 
 
 
 ideal 3 
 
 
 
 
 
 
 
 
 
 P=3 
 
 
 P=3 
 
 
 
 
 
 
 
 
 
 ideal 4 
 
 
 ideal 4

In [63]:
EOCs

[ 0.8866938066352823, 1.9764605264663584, 2.3071025518239012, 2.5978296649439234 ]

Assertion

In [64]:
NUnit.Framework.Assert.IsTrue(EOCs[0]>0.8);

In [65]:
NUnit.Framework.Assert.IsTrue(EOCs[1]>1.8);

In [66]:

NUnit.Framework.Assert.IsTrue(EOCs[2]>2.2);

In [67]:

NUnit.Framework.Assert.IsTrue(EOCs[3]>2.5);

## Plotting the ShadowFields (for visualization)

choose a concrete session

In [57]:
// var sessions = database.Sessions.Where(s => s.Timesteps.Count >0);
// sessions=sessions.Where(s => s.Name.Contains("10x32") && s.Name.Contains("iFlx0"));
// sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0*	5/27/2024 1:20:36 PM	9e3cc0b3...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:06 PM	990fe52f...


plot it

In [58]:
// using BoSSS.Solution.Tecplot;
// using System.IO;
// var si=sessions.Pick(0);
// Directory.GetFiles(".", "*_SP_*").ForEach(file => File.Delete(file));
// IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
// //double[] TStoPlot = new double[] {0,5,10,15,20}; // specify selected Iterations
// var allTs = (List<double>) lts.TimeStepNumbers; 
// var N=5;
// var TStoPlot= allTs.Where(ts => ts % N == 0); // plot every N
// foreach(var timestep in si.Timesteps){
//     if(TStoPlot.Contains((double) timestep.TimeStepNumber.MajorNumber)){
//             IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) timestep).GetInternal();
//             var SF = lts.GetShadowFields();
//             var texplot = new Tecplot(SF[0].GridDat, 2);
//             texplot.PlotFields( "BS_SP_" + lts.TimeStepNumber,0.0,SF);
//     }
// }